# Pengujian dan Prediction Request ke Model Serving

Pada bagian ini, dilakukan proses pengujian dan prediction request kepada model serving menggunakan HTTP request yang berasal dari model yang sudah berhasil di deploy menggunakan platform cloud bernama Railway. Proses ini menetapkan sebuah URL endpoint yang menjadi API utama untuk melakukan permintaan dalam bentuk HTTP request dengan memanggil model serving yang di-host oleh TF-Serving. URL tersebut adalah `https://mlops-disaster-tweets-production.up.railway.app/v1/models/tweets-model:predict` dan dirancang untuk meminta prediksi dari model.

Berdasarkan proses machine learning pipeline yang dibuat, diketahui bahwa format data yang diterima oleh model adalah TFRecord, sedangkan proses permintaan (request) prediksi di production environment harus berupa string JSON (Javascript Object Notation). Beberapa karakter dalam data TFRecord dapat menjadi masalah saat disertakan langsung dalam JSON karena dianggap tidak valid. Oleh karena itu, digunakan teknik pengkodean menggunakan Base64 encoding yang secara umum mewakili data biner, sehingga data biner TFRecord dapat disertakan dengan aman dalam string JSON tanpa resiko terjadinya karakter yang tidak valid dan bisa diterima oleh model.

Berikut adalah kode untuk menjalankan pengujian dan prediction request ke model serving.

### Import Dependencies

In [1]:
import requests
import tensorflow as tf
import base64

### URL TF-Serving

In [16]:
# URL for TF-Serving
serving_url = "https://mlops-disaster-tweets-production.up.railway.app/v1/models/tweets-model:predict"

### Input String Tweet Text Example and ID Tweet

In [43]:
# input string text
id_value = 1
input_string = "Just happened a terrible car crash"

### Data Preprocessing (Encoded the serializede example from TFRecord format)

In [44]:
# create tf.train.Example from the input string
example = tf.train.Example(features=tf.train.Features(feature={
    'id': tf.train.Feature(int64_list=tf.train.Int64List(value=[id_value])),
    'text': tf.train.Feature(bytes_list=tf.train.BytesList(value=[input_string.encode()]))
}))
tfrecord_example = example.SerializeToString()

# base64 encode the serialized example
b64_encoded_example = base64.b64encode(tfrecord_example).decode("utf-8")

# prepare the input example in the correct format
input_example = {"b64": b64_encoded_example}

### Make Prediction Request

In [45]:
print("Tweet:", input_string)

# send prediction request to TF-Serving
response = requests.post(serving_url, json={"instances": [input_example]})

# check the response
if response.status_code == 200:
    result = response.json()
    model_output = result["predictions"][0][0]
    print("\nModel output:", model_output)

    # check if the model output is spam (assuming threshold is 0.75)
    tweet_threshold = 0.75
    if model_output > tweet_threshold:
        print("Prediction: Disaster tweet")
    else:
        print("Prediction: Not Disaster tweet (just normal tweet)")
else:
    print("Error:", response.status_code, response.content)

Tweet: Just happened a terrible car crash

Model output: 0.914532244
Prediction: Disaster tweet


Berdasarkan hasil output dari prediction request, didapat hasil bahwa contoh tweet yang digunakan untuk pengujian merupakan tweet berjenis Disaster Tweet dengan output model sekitar 0.91. Hasil ini menunjukkan bahwa model bekerja cukup baik dalam mengklasifikasikan kalimat tweet.